In [32]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [38]:
df = pd.read_csv("D:\\DataSet\\covid_toy.csv")

In [5]:
df.head(3)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [39]:
x = df.drop(columns = ["has_covid"])
y = df["has_covid"]

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [41]:
x_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
88,5,Female,100.0,Mild,Kolkata
26,19,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi
69,73,Female,103.0,Mild,Delhi
...,...,...,...,...,...
60,24,Female,102.0,Strong,Bangalore
71,75,Female,104.0,Strong,Delhi
14,51,Male,104.0,Mild,Bangalore
92,82,Female,102.0,Strong,Kolkata


## Handling missing value using SimpleImputer

In [16]:
si = SimpleImputer()
x_train_fever = si.fit_transform(x_train[["fever"]])
x_test_fever = si.fit_transform(x_test[["fever"]])
x_train_fever.shape

(80, 1)

## OrdinalEncoding

In [14]:
oe = OrdinalEncoder(categories = [["Mild", "Strong"]])
x_train_cough = oe.fit_transform(x_train[["cough"]])
x_test_cough = oe.fit_transform(x_test[["cough"]])
x_train_cough.shape

(80, 1)

## OneHotEncoder

In [18]:
ohe = OneHotEncoder(drop="first", sparse_output=False, dtype=np.int32)
x_train_gender_city = ohe.fit_transform(x_train[["gender", "city"]])
x_test_gender_city = ohe.fit_transform(x_test[["gender", "city"]])
x_train_gender_city.shape

(80, 4)

In [21]:
x_train_age = x_train.drop(columns = ["gender", "fever", "cough", "city"]).values
x_test_age = x_test.drop(columns = ["gender", "fever", "cough", "city"]).values
x_train_age.shape

(80, 1)

In [22]:
x_train_transformed = np.concatenate((
    x_train_age,
    x_train_fever,
    x_train_gender_city,
    x_train_cough
), axis=1)

In [23]:
x_train_transformed.shape

(80, 7)

# By the help of columns transformer

In [27]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[

    ("tnf1", SimpleImputer(), ["fever"]),
    ("tnf2", OrdinalEncoder(categories = [["Mild", "Strong"]]), ["cough"]),
    ("tnf3", OneHotEncoder(drop="first", sparse_output=False), ["gender", "city"])
    
], remainder="passthrough")

In [28]:
transformer.fit_transform(x_train).shape

(80, 7)

In [30]:
transformer.transform(x_test).shape

(20, 7)

In [45]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers=[

    ("tnf1", SimpleImputer(), ["fever"]),
    ("tnf2", LabelEncoder(), ["cough"]),
    ("tnf3", pd.get_dummies(df, drop_first=True), ["gender", "city"])
    
], remainder="passthrough")

In [49]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('tnf1', SimpleImputer(), ['fever']),
                                ('tnf2', LabelEncoder(), ['cough']),
                                ('tnf3',
                                     age  fever  gender_Male  cough_Strong  city_Delhi  city_Kolkata  \
0    60  103.0         True         False       False          True   
1    27  100.0         True         False        True         False   
2    42  101.0         True         False        True         False   
3    31   98.0        False         False       False          True   
4    65  101.0        False         False       False         False   
..  ...    ...          ...           ...         ...           ...   
95   12  104.0        False         False       False         False   
96   51  101.0        False          True       False          True   
97   20  101.0        False         False       False         False   
98    5   98.0        False          True       False         False   
99   10   98.0        False          True       False          True   

    city_Mumbai  has_covid_Yes  
0         False          False  
1         False           True  
2         False          False  
3         False          False  
4          True          False  
..          ...            ...  
95        False          False  
96        False           True  
97        False          False  
98         True          False  
99        False           True  

[100 rows x 8 columns],
                                 ['gender', 'city'])])